In [1]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 811.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4
    Uninstalling networkx-3.4:
      Successfully uninstalled networkx-3.4
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.0.1
    Uninstalling geopandas-1.0.1:
      Successfully uninstalled geopandas-1.0.1


In [2]:
import heapq

# Lista de capacidades de cada nodo
capacidades_nodos = {'A': 33, 'B': 50, 'C': 63, 'D': 15, 'E': 0, 'F': 90, 'G': 91, 'H': 40, 'I': 23}

# Lista de distancias entre nodos de un punto a otro teniendo en cuenta que (a,b) no es lo mismo que (b,a)
distancias_renombradas = {
    'X': {'A': 4289.39, 'B': 2887.56, 'C': 3349.81, 'D': 2044.60, 'E': 2439.43, 'F': 873.80, 'G': 1388.11, 'H': 678.69, 'I': 547.35},
    'A': {'X': 4033.17, 'B': 2417.92, 'C': 1586.21, 'D': 3467.37, 'E': 3284.98, 'F': 3182.96, 'G': 5159.82, 'H': 4437.55, 'I': 4080.41},
    'B': {'X': 3225.05, 'A': 3178.66, 'C': 1654.33, 'D': 1978.83, 'E': 1793.74, 'F': 3033.58, 'G': 3729.18, 'H': 3006.91, 'I': 3352.13},
    'C': {'X': 3332.11, 'A': 1524.33, 'B': 1458.13, 'D': 2537.72, 'E': 2352.63, 'F': 2481.91, 'G': 4288.07, 'H': 3565.80, 'I': 3379.36},
    'D': {'X': 2449.06, 'A': 3685.67, 'B': 2035.39, 'C': 2558.60, 'E': 394.82, 'F': 2261.25, 'G': 2234.64, 'H': 1985.24, 'I': 2576.14},
    'E': {'X': 3197.52, 'A': 3522.90, 'B': 1872.62, 'C': 2395.83, 'D': 1409.83, 'F': 3009.71, 'G': 3418.46, 'H': 2818.92, 'I': 3324.60},
    'F': {'X': 882.09, 'A': 3415.59, 'B': 3097.73, 'C': 2476.01, 'D': 2257.29, 'E': 2652.11, 'G': 2151.32, 'H': 1560.78, 'I': 929.33},
    'G': {'X': 1572.22, 'A': 5861.61, 'B': 4459.78, 'C': 4922.03, 'D': 3616.82, 'E': 4011.65, 'F': 2446.02, 'H': 920.03, 'I': 1699.30},
    'H': {'X': 684.18, 'A': 4973.57, 'B': 3571.74, 'C': 4033.99, 'D': 2728.79, 'E': 3123.61, 'F': 1557.98, 'G': 1188.32, 'I': 811.26},
    'I': {'X': 547.35, 'A': 4344.92, 'B': 3434.91, 'C': 3405.34, 'D': 2591.96, 'E': 2986.78, 'F': 929.33, 'G': 1515.19, 'H': 805.77}
}

# Capacidad máxima del camión recolector
capacidad_maxima = 1000

# Normalización de valores
max_capacidad = max(capacidades_nodos.values())
min_capacidad = min(capacidades_nodos.values())

def normalizar(valor, max_valor, min_valor):
    return (valor - min_valor) / (max_valor - min_valor)

def seleccionar_siguiente_nodo(nodo_actual, distancias, capacidades, visitados, capacidad_restante):
    max_heap = []  # Usamos un max_heap para prioridades invertidas
    print(f"\nAnalizando opciones desde el nodo {nodo_actual}...")
    for nodo_destino, distancia in distancias[nodo_actual].items():
        if nodo_destino not in visitados and nodo_destino != 'X' and capacidades[nodo_destino] > 0:  # Excluir nodos con capacidad 0
            distancia_normalizada = normalizar(distancia, max(distancias[nodo_actual].values()), min(distancias[nodo_actual].values()))
            capacidad_normalizada = normalizar(capacidades[nodo_destino], max_capacidad, min_capacidad)
            # Ajustamos la prioridad para dar más peso a la capacidad
            prioridad = (0 * capacidad_normalizada) + (1 * (1 - distancia_normalizada))

            print(f"  Considerando nodo {nodo_destino} con capacidad {capacidades[nodo_destino]} y distancia {distancia:.2f} metros. Prioridad: {prioridad:.2f}")

            if capacidades[nodo_destino] <= capacidad_restante:
                # Insertamos con prioridad negativa para simular un max_heap
                heapq.heappush(max_heap, (-prioridad, nodo_destino, distancia))
            else:
                print(f"  Nodo {nodo_destino} excede la capacidad restante del camión. No se considera.")

    if max_heap:
        # Extraemos con prioridad negativa para obtener el máximo real
        mejor_opcion = heapq.heappop(max_heap)
        mejor_opcion = (-mejor_opcion[0], mejor_opcion[1], mejor_opcion[2])  # Convertimos de nuevo la prioridad
        print(f"  Seleccionado nodo {mejor_opcion[1]} con prioridad {mejor_opcion[0]:.2f}, capacidad {capacidades[mejor_opcion[1]]} y distancia {mejor_opcion[2]:.2f} metros.")
        return mejor_opcion[1], capacidades[mejor_opcion[1]], mejor_opcion[2]

    print("  No hay más nodos disponibles para visitar o todos exceden la capacidad restante del camión.")
    return None, 0, 0

def recorrido_priorizado(nodo_inicial, distancias, capacidades, capacidad_maxima):
    nodo_actual = nodo_inicial
    visitados = set([nodo_actual])
    recorrido = [nodo_actual]
    capacidad_restante = capacidad_maxima
    distancia_total = 0
    print(f"Iniciando en el nodo {nodo_actual}. Capacidad restante: {capacidad_restante}")

    while True:
        siguiente_nodo, capacidad_nodo, distancia = seleccionar_siguiente_nodo(nodo_actual, distancias, capacidades, visitados, capacidad_restante)
        if siguiente_nodo is None:
            print("No hay más nodos disponibles para visitar o todos exceden la capacidad restante del camión. Finalizando recorrido.")
            break
        visitados.add(siguiente_nodo)
        recorrido.append(siguiente_nodo)
        capacidad_restante -= capacidad_nodo
        distancia_total += distancia
        print(f"Moviéndose al nodo {siguiente_nodo} con capacidad {capacidades[siguiente_nodo]}. Capacidad restante: {capacidad_restante}. Distancia total recorrida: {distancia_total:.2f} metros.")
        nodo_actual = siguiente_nodo
        if capacidad_restante <= 0:
            print("El camión ha alcanzado su capacidad máxima. Finalizando recorrido.")
            break

    if nodo_actual != nodo_inicial:
        distancia_de_regreso = distancias[nodo_actual][nodo_inicial]
        distancia_total += distancia_de_regreso
        recorrido.append(nodo_inicial)
        print(f"Regresando al nodo {nodo_inicial} desde el nodo {nodo_actual}. Distancia de regreso: {distancia_de_regreso:.2f} metros.")
        print(f"Distancia total recorrida al regresar al nodo inicial: {distancia_total:.2f} metros")

    print(f"\nRecorrido completo: {recorrido}")
    print(f"Distancia total recorrida: {distancia_total:.2f} metros")
    return recorrido, distancia_total, capacidad_restante

nodo_inicial = 'X'
recorrido, distancia_total, capacidad_restante = recorrido_priorizado(nodo_inicial, distancias_renombradas, capacidades_nodos, capacidad_maxima)

print(f"\nRecorrido completo: {recorrido}")
print(f"Distancia total recorrida: {distancia_total:.2f} metros")
print(f"Capacidad restante: {capacidad_restante}")


Iniciando en el nodo X. Capacidad restante: 1000

Analizando opciones desde el nodo X...
  Considerando nodo A con capacidad 33 y distancia 4289.39 metros. Prioridad: 0.00
  Considerando nodo B con capacidad 50 y distancia 2887.56 metros. Prioridad: 0.37
  Considerando nodo C con capacidad 63 y distancia 3349.81 metros. Prioridad: 0.25
  Considerando nodo D con capacidad 15 y distancia 2044.60 metros. Prioridad: 0.60
  Considerando nodo F con capacidad 90 y distancia 873.80 metros. Prioridad: 0.91
  Considerando nodo G con capacidad 91 y distancia 1388.11 metros. Prioridad: 0.78
  Considerando nodo H con capacidad 40 y distancia 678.69 metros. Prioridad: 0.96
  Considerando nodo I con capacidad 23 y distancia 547.35 metros. Prioridad: 1.00
  Seleccionado nodo I con prioridad 1.00, capacidad 23 y distancia 547.35 metros.
Moviéndose al nodo I con capacidad 23. Capacidad restante: 977. Distancia total recorrida: 547.35 metros.

Analizando opciones desde el nodo I...
  Considerando nodo A 

In [3]:
import osmnx as ox
import folium
import heapq
import networkx as nx
from folium.plugins import MarkerCluster
from IPython.display import display, HTML
import branca
import heapq
import networkx as nx


# Nombre de la ciudad
nombre_ciudad = "San Borja, Peru"

# Obtiene el grafo completo de la ciudad
grafo_completo = ox.graph_from_place(nombre_ciudad, network_type='drive')

# Lista de nodos camino
nodos_camino = [473980952, 485629985, 4354035769, 485617722, 485617723, 4354035770, 485615688, 473974857,
                485615690, 485615691, 485615693, 473974863, 4351514723, 485617798, 4324429964, 326021261,
                485626003, 316625051, 473974957, 1273933999, 485617839, 485617841, 485617843, 493916343,
                4042528362, 1641423047, 1641423050, 316625114, 316625118, 3628181742, 412557570, 419481861,
                419481862, 412557576, 419481865, 419481866, 4549916939, 419481867, 4549916938, 419481871,
                412557584, 419481873, 419481874, 419481876, 419481877, 419481879, 3628169497, 419481882,
                419481883, 412557600, 419481888, 419481893, 3628169514, 419481899, 419481900, 473977131,
                4069101876, 515494213, 4351533382, 419481936, 4351533393, 515494224, 4351533396, 4351533399,
                515494237, 4267524448, 1641255311, 417251728, 417251733, 1641255321, 1641255330, 485632418,
                485618096, 417251778, 417251793, 417251806, 316627523, 316627529, 1273662037, 1348442714,
                1348442735, 485622400, 417249925, 4353006213, 412553899, 4354632363, 412553905, 485616311,
                1273459387, 1047200452, 3628219077, 412553924, 485618375, 1047200456, 3628219078, 3628219079,
                1047200460, 1047200465, 6277866193, 412553943, 316627676, 1047200478, 1047200484, 392151788,
                3630203630, 6408227575, 1641423619, 412553989, 1641423622, 417252101, 417252106, 412553995,
                417252107, 417252121, 417252124, 412554019, 417252136, 417252137, 417252139, 412554037,
                412554042, 412554048, 412554053, 4170789705, 412554058, 1273920329, 391037794, 4037364582,
                4037364583, 1273535339, 1273547639, 392153976, 412554131, 412554148, 392154024, 485624745,
                4109970355, 1641249721, 1641249722, 1641249723, 1641249724, 1641249725, 1641249727, 791159755,
                4808012769, 3630330855, 3630330864, 4022602745, 495672319, 419482625, 4549868556, 791159829,
                485618714, 412554266, 791159835, 326022171, 412554277, 391037994, 4069186616, 391038008,
                412554299, 528538687, 791159875, 471608390, 412554319, 412554325, 791159897, 10363683930,
                412554336, 791159916, 471581808, 1903617139, 412554359, 412554365, 263928959, 5523457156,
                1623276677, 3630222472, 3630222483, 791159958, 412554411, 485616817, 485616822, 485616827,
                791159998, 391038143, 391038145, 391038146, 391038147, 412529863, 2329670857, 485616841,
                791160018, 417252565, 417252566, 417252567, 417252568, 485616856, 417252570, 417252571,
                417252572, 4357999837, 2329670874, 2329670877, 485616864, 412529888, 412529885, 5354534115,
                412529892, 412529886, 412529887, 417252576, 417252579, 5599098090, 485620979, 392152310,
                392152312, 392152314, 392152316, 791160060, 392152318, 392152319, 392152320, 392152321,
                392152322, 392152323, 392152324, 791160064, 392152327, 485616903, 485616904, 10001175828,
                10001175829, 471581977, 4115602736, 4115602737, 1273664820, 1273650490, 391038288, 391038294,
                391038295, 417252706, 791160171, 419483012, 791160201, 791160207, 791160212, 2990124458,
                2990124459, 4356640171, 3949426095, 3949426097, 485633467, 417250755, 4022717906, 4022717907,
                4022717915, 791160283, 263935452, 412529880, 417250837, 411307544, 391038491, 391038492,
                411307550, 417250848, 391038498, 391038513, 4374564406, 485619262, 316069449, 485619279,
                1641248337, 485619286, 263108188, 412528226, 392152681, 4042528361, 412528235, 392152684,
                392152685, 392152686, 1641248367, 392152687, 392152688, 392152689, 392152691, 1641248372,
                392152692, 392152693, 412528242, 392152696, 392152697, 412528246, 412528254, 1641248384,
                4037293700, 485615244, 485615245, 1641248403, 412528280, 412528281, 412528284, 412528293,
                412528294, 412528295, 412528296, 1641248425, 412528297, 412528299, 1641248429, 412528303,
                411307701, 1641248438, 412528311, 9655203518, 412528319, 412528320, 412528321, 3630229213,
                3630229215, 3630229218, 3630229223, 6302766843, 6302766844, 2383679233, 2383679236, 2383679238,
                4354637579, 1047201560, 316069659, 1047201571, 1047201573, 1047201579, 1047201588, 1047201589,
                417251126, 417251128, 417251133, 417251134, 417251135, 417251136, 417251137, 4022495044,
                391036740, 1841059656, 4069103434, 1841059660, 417251154, 417251165, 417251169, 391036770,
                316624764, 485617542, 417251208, 485617546, 391036814, 485617552, 3903819668, 263933848,
                263933849, 263933850, 263933851, 316071836, 263933854, 391036861, 528535487, 4352989148,
                4352989150, 485615583, 4352989151, 11346851818, 11346851819, 11346851820, 11346851821]

# Lista de nodos a resaltar
nodos_resaltados = [3628169514, 1641248425, 1047200484, 791160018, 412554058, 392153976, 392152310, 2329670857, 4549916939, 417252571]

# Nombres de los nodos resaltados (pueden ser cambiados según se necesite)
nombres_nodos_resaltados = {
    3628169514: "X",
    1641248425: "A",
    1047200484: "B",
    791160018: "C",
    412554058: "D",
    392153976: "E",
    392152310: "F",
    2329670857: "G",
    4549916939: "H",
    417252571: "I"
}

# Crea el subgrafo con los nodos del camino
grafo_filtrado = grafo_completo.subgraph(nodos_camino)

# Crea un mapa centrado en San Borja, Perú
m = folium.Map(location=[-12.10129, -77.02089], zoom_start=15)

# Añade marcadores y etiquetas para los nodos resaltados
for nodo, nombre in nombres_nodos_resaltados.items():
    lat = grafo_filtrado.nodes[nodo]['y']
    lon = grafo_filtrado.nodes[nodo]['x']
    color = 'red' if nombre == 'X' else 'blue'
    folium.Marker(
        [lat, lon],
        icon=folium.Icon(color=color),
        popup=folium.Popup(nombre, parse_html=True)
    ).add_to(m)

# Añade líneas para los nodos camino
for u, v, data in grafo_filtrado.edges(data=True):
    lat_u = grafo_filtrado.nodes[u]['y']
    lon_u = grafo_filtrado.nodes[u]['x']
    lat_v = grafo_filtrado.nodes[v]['y']
    lon_v = grafo_filtrado.nodes[v]['x']
    folium.PolyLine(locations=[(lat_u, lon_u), (lat_v, lon_v)], color='gray').add_to(m)

# Colores para cada segmento del camino
colores_segmentos = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'pink', 'black']



# Función que encuentra la ruta y la traza basado en los nodos caminos
def trazar_ruta(nodos):
    ruta = []
    for i in range(len(nodos) - 1):
        origen = list(nombres_nodos_resaltados.keys())[list(nombres_nodos_resaltados.values()).index(nodos[i])]
        destino = list(nombres_nodos_resaltados.keys())[list(nombres_nodos_resaltados.values()).index(nodos[i+1])]
        path = nx.shortest_path(grafo_filtrado, source=origen, target=destino, weight='length')
        ruta.extend(path[:-1])  # Evitar duplicados en la unión de caminos
    ruta.append(list(nombres_nodos_resaltados.keys())[list(nombres_nodos_resaltados.values()).index(nodos[-1])])
    return ruta


if recorrido:
    # Añade la ruta al mapa con segmentos de diferentes colores
    nodos_ruta = recorrido
    for i in range(len(nodos_ruta) - 1):
        origen = list(nombres_nodos_resaltados.keys())[list(nombres_nodos_resaltados.values()).index(nodos_ruta[i])]
        destino = list(nombres_nodos_resaltados.keys())[list(nombres_nodos_resaltados.values()).index(nodos_ruta[i+1])]
        path = nx.shortest_path(grafo_filtrado, source=origen, target=destino, weight='length')
        for j in range(len(path) - 1):
            lat_u = grafo_filtrado.nodes[path[j]]['y']
            lon_u = grafo_filtrado.nodes[path[j]]['x']
            lat_v = grafo_filtrado.nodes[path[j+1]]['y']
            lon_v = grafo_filtrado.nodes[path[j+1]]['x']
            folium.PolyLine(locations=[(lat_u, lon_u), (lat_v, lon_v)], color=colores_segmentos[i % len(colores_segmentos)], weight=5, opacity=0.8).add_to(m)

    # Genera la leyenda que será actualizada dinámicamente
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 250px; height: auto;
                background-color: white; z-index:9999; font-size: 14px;
                border:2px solid grey; padding: 10px;">
        <b>Ruta a Seguir</b><br>
    '''

    for i in range(len(nodos_ruta) - 1):
        legend_html += f'''
        <i style="background: {colores_segmentos[i % len(colores_segmentos)]}; width: 10px; height: 10px; float: left; margin-right: 5px;"></i> {nodos_ruta[i]} a {nodos_ruta[i+1]}<br>
        '''

    legend_html += '</div>'

    m.get_root().html.add_child(folium.Element(legend_html))

    # Guarda y muestra el mapa que está en html
    m.save('nodos_mapa.html')
    m
else:
    print("No se generó ningún mapa porque todos los tachos están vacíos.")

In [6]:
m